### Clean the Local Area Unemployment Statistics data
Data pulled from The Bureau of Labor Statistics' (BLS) Public Data Application Programming Interface (API)

In [3]:
import pandas as pd

In [4]:
# Read in the data 
df = pd.read_csv('./all_data_by_city.csv')

In [5]:
df.head()

,series_id,date,year,period,value,city,series_type
0,LAUCS090731000000003,07/2020,2020,M07,9.1,"Branford town, CT",unemployment_rate
1,LAUCS090731000000003,06/2020,2020,M06,9.1,"Branford town, CT",unemployment_rate
2,LAUCS090731000000003,05/2020,2020,M05,8.9,"Branford town, CT",unemployment_rate
3,LAUCS090731000000003,04/2020,2020,M04,8.0,"Branford town, CT",unemployment_rate
4,LAUCS090731000000003,03/2020,2020,M03,3.1,"Branford town, CT",unemployment_rate


In [7]:
# Change column name to better describe the city info provided
df.rename(columns={"city": "city_type_state"}, inplace=True)

In [9]:
# Convert city information to separate city, state, and city_state columns
cities = []
states = []
cities_states = []
for item in df['city_type_state']:
    city, state = item.split(', ')
    city = ' '.join(city.split(' ')[:-1])
    city_state = city+', '+state
    cities.append(city)
    states.append(state)
    cities_states.append(city_state)
    
df['city'] = cities
df['state'] = states
df['city_state'] = cities_states

In [10]:
df.head()

,series_id,date,year,period,value,city_type_state,series_type,city,state,city_state
0,LAUCS090731000000003,07/2020,2020,M07,9.1,"Branford town, CT",unemployment_rate,Branford,CT,"Branford, CT"
1,LAUCS090731000000003,06/2020,2020,M06,9.1,"Branford town, CT",unemployment_rate,Branford,CT,"Branford, CT"
2,LAUCS090731000000003,05/2020,2020,M05,8.9,"Branford town, CT",unemployment_rate,Branford,CT,"Branford, CT"
3,LAUCS090731000000003,04/2020,2020,M04,8.0,"Branford town, CT",unemployment_rate,Branford,CT,"Branford, CT"
4,LAUCS090731000000003,03/2020,2020,M03,3.1,"Branford town, CT",unemployment_rate,Branford,CT,"Branford, CT"


In [12]:
# Check for missing city name values
df[df['city']=='']['city_type_state'].value_counts() # display city_type_state for cities missing a name

Princeton, NJ    364
Name: city_type_state, dtype: int64

In [8]:
df['city_state'] = df['city_state'].replace(', NJ', 'Princeton, NJ')
df['city'] = df['city'].replace('', 'Princeton')

In [9]:
# Verify that there are no remaining missing city values
df[df['city']=='']

,series_id,date,year,period,value,city,series_type,city_type_state,state,city_state


In [15]:
# Many MA towns had an extra "Town" specification that was unrecognized by Tableau
df[df['city'].str.contains('Town')]['city_type_state'].value_counts()

Franklin Town city, MA            460
Methuen Town city, MA             460
Watertown Town city, MA           460
Agawam Town city, MA              460
West Springfield Town city, MA    460
Braintree Town city, MA           460
Barnstable Town city, MA          460
Weymouth Town city, MA            460
Name: city_type_state, dtype: int64

In [12]:
# Remove extra "Town" specification
cleaned_cities = []
for city in df['city'].values:
    city = city.replace(' Town', '')
    cleaned_cities.append(city)
df['city'] = cleaned_cities

In [13]:
# Verify that "Town" specification was removed
df[df['city'].str.contains('Town')]

,series_id,date,year,period,value,city,series_type,city_type_state,state,city_state


In [16]:
# Many of the listed cities had additional information in Parenthesis
df[df['city'].str.contains(r"\s?\(")]['city_type_state'].value_counts()

Bedford township (Monroe County), MI                   460
Washington township (Gloucester County), NJ            460
Montgomery township (Montgomery County), PA            460
Spring township (Berks County), PA                     460
Plainfield charter township (Kent County), MI          460
Butte-Silver Bow (consolidated) city, MT               460
Milford (consolidated) city/town, CT                   460
Shelby charter township (Macomb County), MI            460
Brighton town (Monroe County), NY                      460
Columbus (consolidated) city, GA                       460
Clinton charter township (Macomb County), MI           460
Bloomfield charter township (Oakland County), MI       460
Middletown township (Bucks County), PA                 460
Derry township (Dauphin County), PA                    460
Franklin township (Somerset County), NJ                460
Washington township (Macomb County), MI                460
Falls township (Bucks County), PA                      4

In [15]:
# Remove information in parentheses from city data and reassign to 
city_parentheses = list(set(df[df['city'].str.contains(r"\s?\(")]['city_type_state']))

city_rm_parentheses = [city.split(', ')[0].split(r" (")[0] for city in city_parentheses]

city_parentheses_dict = {} # make dictionary of cities as listed to cleaned city names
for i in range(len(city_parentheses)):
    key = city_parentheses[i].split(', ')[0]
    key = ' '.join(key.split(' ')[:-1])
    value = city_rm_parentheses[i]
    city_parentheses_dict[key] = value

all_cities_no_parentheses = [] # update values of the city column
for city in df['city']:
    if city in city_parentheses_dict:
        all_cities_no_parentheses.append(city_parentheses_dict[city])
    else:
        all_cities_no_parentheses.append(city)
df['city'] = all_cities_no_parentheses

In [16]:
# Verify that no city names remain with extra info in parentheses
df[df['city'].str.contains(r"\s?\(")]

,series_id,date,year,period,value,city,series_type,city_type_state,state,city_state


In [17]:
df.to_csv('./cleaned_bls_data.csv')